In [7]:
import argparse 
import ast

import cf_xarray
#import cftime
import geocat.comp as gcomp
#import holoviews as hv
#import hvplot
#import hvplot.xarray
#import intake
import numpy as np
#import pop_tools
import xarray as xr
import xesmf as xe

#from distributed import Client
#from ncar_jobqueue import NCARCluster
#from pop_tools.grid import _compute_corners

import logging 
import netCDF4 as nc

## Questions: 
Do certain variables still need to be reversed?  
Is surface geopotential and lowest layer geopotential treated the same here? 
Does geocat's interp function work across time? 
TO DO:  
Implement weight reuse  
Implement pressure levels   
Fix wrong standard name for SST  
Add hooks for user-defined reference pressure?  
Fix up METGRID default value  
Implement unit converter (pint) or unit checker  
NOTES:   
Created a branch for a version that works on cf-compliant data  
 


In [8]:
#Command line option handling ----------------------------------------------------------------------------------
parser = argparse.ArgumentParser()  
logging.basicConfig(level=logging.DEBUG)
current_log_level = logging.getLogger().getEffectiveLevel() 

parser.add_argument('CASE',type=str, help='One of the following IPCC Climate Scenarios: 20THC/RCP85/RCP60/RCP45')
parser.add_argument('--o',type=str,help='Output directory path')
parser.add_argument('--mode','-m',type=str,help='Set logging mode: DEBUG/INFO/WARNING/ERROR/CRITICAL')
parser.add_argument('--plev',type=str, help="File name of desired output pressure levels")
parser.add_argument('--weights',type=str, help="File name if reusing regridding weights")

if current_log_level != 10: 
    args = parser.parse_args()

In [9]:
#File Handling ----------------------------------------------------------------------------------

logging.info("Opening data files...")

in_ta = xr.open_dataset("atmos_ta.nc")         # 6-hourly 3-d T
in_ua = xr.open_dataset("atmos_ua.nc")         # 6-hourly 3-d U
in_va = xr.open_dataset("atmos_va.nc")         # 6-hourly 3-d V
in_hus = xr.open_dataset("atmos_hus.nc")       # 6-hourly 3-d Q
in_ps = xr.open_dataset("atmos_ps.nc")         # 6-hourly surface pressure
in_zsfc = xr.open_dataset("atmos_zsfc.nc")     # static surface geopotential
in_lmask = xr.open_dataset("atmos_lmask.nc")   # static land mask
in_snw = xr.open_dataset("atmos_snw_1.nc")     # monthly SWE
in_mrlsl = xr.open_dataset("atmos_mrlsl_1.nc") # monthly soil moisture
in_ts = xr.open_dataset("atmos_ts_1.nc")       # monthly skin temp
in_tsl = xr.open_dataset("atmos_tsl_1.nc")     # monthly soil temp
in_tos = xr.open_dataset("atmos_tos_1.nc")     # daily SST on pop grid (gaussian)
in_sic = xr.open_dataset("atmos_sic_1.nc")     # daily SEAICE % on POP grid (gaussian)

INFO:root:Opening data files...


In [10]:
#Regrid SST and SEA ICE fields to CESM Atmospheric Domain ----------------------------------------------------------------------------------

logging.info('Converting Parallel Ocean Program data to coordinate system of atmospheric grid...')

SST = in_tos.cf['surface_temperature']
#Create a mask (not needed for interpolating to atmospheric grid, but just in case there are missing values)
#NOTE THAT THIS CF REFERENCE IS WRONG. SST IS THE CORRECT STANDARD NAME WHICH NEEDS TO BE CORRECTED IN THE DATA
in_tos["mask"] = ~SST.cf.isel(time=0).isnull()

#Regrids SST grid to whatever the atmospheric grid is automatically
regrid = xe.Regridder(in_tos, in_ta, method = 'bilinear', periodic=True, unmapped_to_nan=True)
regrid.to_netcdf('weights_gx1v6_latlon.nc') #write out weights for reuse 

regridded_SST = regrid(in_tos)

if current_log_level == 10: 
    print(regridded_SST)
#regridded_SST.to_netcdf('python_regrid.nc')
#
#use some sort of broadcasting or view here to clone to a 6-hrly variable

INFO:root:Converting Parallel Ocean Program data to coordinate system of atmospheric grid...
/glade/work/wukenton/conda-envs/python_correct/lib/python3.9/site-packages/cf_xarray/accessor.py:1772: UserWarning: Variables {'areacello'} not found in object but are referred to in the CF attributes.
  warnings.warn(
/glade/work/wukenton/conda-envs/python_correct/lib/python3.9/site-packages/xesmf/smm.py:130: UserWarning: Input array is not C_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not C_CONTIGUOUS. ' 'Will affect performance.')


<xarray.Dataset>
Dimensions:   (vertices: 4, lat: 192, lon: 288, time: 92)
Coordinates:
  * time      (time) object 2006-10-01 12:00:00 ... 2006-12-31 12:00:00
  * lat       (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Dimensions without coordinates: vertices
Data variables:
    lon_bnds  (vertices, lat, lon) float32 nan nan nan nan ... 248.7 248.7 248.7
    lat_bnds  (vertices, lat, lon) float32 nan nan nan nan ... 89.65 89.65 89.65
    tos       (time, lat, lon) float32 nan nan nan nan ... 271.3 271.3 271.3
    mask      (lat, lon) bool True True True True True ... True True True True
Attributes:
    regrid_method:  bilinear


In [11]:
#Prepare Variables for Interpolation ----------------------------------------------------------------------------------
hyam = in_ta.cf['hyam'] 
hybm = in_ta.cf['hybm']
hyai = in_ta.cf['hyai']
hybi = in_ta.cf['hybi']

surf_pressure = in_ps.cf['PS']

def psl_ecmwf(temp_bottom: xr.DataArray, phi_surf: xr.DataArray, pressure_surf: xr.DataArray, pressure_bot: xr.DataArray): 
    #Based on the NCAR Technical Note "Vertical Interpolation and Truncation of Model-Coordinate Data"
    #By Trenberth, Berry, Buja; Dec 1993 
    #temp_surf = T_*, temp_bottom = T_NL, pressure_surf = p_s, pressure_bot = p_NL, phi_surf = \phi_s

    LAPSE_RATE = 0.0065     #Kelvin per meter
    GRAV_CONST = 9.80616    #Meters per second per second
    SPEC_GAS_CONST = 287.04 #Joules per kilogram per Kelvin

    ALPHA_0 = LAPSE_RATE*SPEC_GAS_CONST/GRAV_CONST
    
    temp_surf = temp_bottom + ALPHA_0*temp_bottom*(pressure_surf/pressure_bot - 1) #3b.5
    
    temp_bot_lapse = temp_surf + LAPSE_RATE*phi_surf/GRAV_CONST #denoted T_0 in doc, 3b.13
    
    #Create a case mask (doing this to avoid using xr.where, which has no short circuit)
    where_case = np.zeros_like(pressure_surf, dtype=np.ubyte)
    where_case[temp_surf <= 290.5 & temp_bot_lapse > 290.5]   = 1
    where_case[temp_surf > 290.5 & temp_bot_lapse > 290.5]    = 2
    where_case[temp_surf < 255]                               = 3

    psl = np.empty_like(pressure_surf)
    for num in [0,1,2,3]: 
        ps = pressure_surf[where_case==num] 
        T_star = temp_surf[where_case==num]
        phi_s = phi_surf[where_case==num]
        match num : 
            case 0: alphas = ALPHA_0
            case 1: alphas = SPEC_GAS_CONST/phi_s*(290.5-T_star) 
            case 3: alphas = 0
            case 4: alphas = ALPHA_0
        combo_term = phi_s/SPEC_GAS_CONST/T_star

        psl[where_case==0] = ps*np.exp(combo_term*(1-1/2*alphas*combo_term+1/3*(alphas*combo_term)**2)) #3b.12
    

    ## may be able to cheapen this operation by implementing some soft of short circuit behavior...

    pressure_mean_sea_level = xr.where(np.absolute(phi_sfc/GRAV_CONST) < 1e-4, pressure_surf, pressure_mean_sea_level)






default_levs = np.array([1000.0, 975.0, 950.0, 925.0, 900.0, 850.0, 800.0, 750.0, 700.0, 650.0, 600.0, 550.0, 500.0, \
             450.0, 400.0, 350.0, 300.0, 250.0, 200.0, 150.0, 100.0, 70.0, 50.0, 30.0, 20.0, 10.0 ])

temp = in_ta["T"]

In [12]:
#Interpolate to Pressure Coordinates ----------------------------------------------------------------------------------
logging.info("Interpolating variables to pressure coordinates...")

if current_log_level == 10: print(temp); print(surf_pressure); print(in_zsfc['PHIS']); 

temp_interp = gcomp.interpolation.interp_hybrid_to_pressure(temp.isel(time=0),surf_pressure.isel(time=0),hyam,hybm, 
                                                            new_levels=default_levs, 
                                                            lev_dim = 'lev', 
                                                            method='log',
                                                            extrapolate=True,
                                                            variable='temperature',
                                                            t_bot=temp.isel(lev=-1,time=0),
                                                            phi_sfc=in_zsfc['PHIS'])

INFO:root:Interpolating variables to pressure coordinates...


<xarray.DataArray 'T' (time: 368, lev: 26, lat: 192, lon: 288)>
[529072128 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lev      (lev) float64 3.545 7.389 13.97 23.94 ... 867.2 929.6 970.6 992.6
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 2006-10-01 00:00:00 ... 2006-12-31 18:00:00
Attributes:
    units:      K
    long_name:  Temperature
<xarray.DataArray 'PS' (time: 368, lat: 192, lon: 288)>
[20348928 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) object 2006-10-01 00:00:00 ... 2006-12-31 18:00:00
Attributes:
    units:      Pa
    long_name:  Surface pressure
<xarray.DataArray 'PHIS' (lat: 192, lon: 288)>
[55296 values with dtype=float64]
Coordinates:
  * lat      (lat) float64

ValueError: cannot align objects with join='exact' where index/labels/sizes are not equal along these coordinates (dimensions): 'lat' ('lat',)